In [1]:
import requests

# Données des objets perdus

In [2]:
def specific_url_lostitem(annee,mois):
    URL = "https://ressources.data.sncf.com/api/records/1.0/search/"
    ressource = "?dataset=objets-trouves-restitution&q=&rows=10000&sort=-date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c&refine.date="
    month_format = f"{annee}%2F{mois}"
    return URL + ressource + month_format

# try_request = requests.get(specific_url(2021,12))

In [3]:
from create_model import LostItem
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import json


engine = create_engine('sqlite:///db.sqlite')
Session = sessionmaker(bind=engine)
session = Session()


for annee in ["2016","2017","2018","2019","2020","2021"]:
    for mois in ["01","02","03","04","05","06","07","08","09","10","11","12"]:
        with open(f'json/data_sncf_{annee}-{mois}.json') as mon_fichier:
            data = json.load(mon_fichier)
        
            for lost_item in data["records"]:
                

                try:
                    date_restitution= lost_item["fields"]["gc_obo_date_heure_restitution_c"]
                except KeyError:
                    date_restitution=None

                try:
                    code_uic_gare_origine= lost_item["fields"]["gc_obo_gare_origine_r_code_uic_c"] 
                except KeyError:
                    code_uic_gare_origine= None   
                
                try:
                    gc_obo_gare_origine_r_name= lost_item["fields"]["gc_obo_gare_origine_r_name"] 
                except KeyError:
                    gc_obo_gare_origine_r_name= None  

                session.add(LostItem(
                    id= lost_item["recordid"] , 
                    code_uic_gare_origine= code_uic_gare_origine ,
                    date= lost_item["fields"]["date"],
                    type_objet=lost_item["fields"]["gc_obo_type_c"] ,
                    gare= gc_obo_gare_origine_r_name,
                    date_restitution=date_restitution 
                    ))


            session.commit()



# Données des gares

In [ ]:
from create_model import Gare
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine


engine = create_engine('sqlite:///db.sqlite')
Session = sessionmaker(bind=engine)
session = Session()

url_game = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=referentiel-gares-voyageurs&q=&rows=10000"
my_request = requests.get(url_game)

field_list = [
    ["code_uic", "uic_code"],
    ["nom_gare", "gare_alias_libelle_noncontraint"],
    ["code_commune", "commune_code"],
    ["code_postal", "adresse_cp"],
    ["code_departement", "departement_numero"],
    ["departement", "departement_libellemin"],
    ["longitude", "longitude_entreeprincipale_wgs84"],
    ["latitude", "latitude_entreeprincipale_wgs84"]
]

for gare in my_request.json()["records"]:
    gare_data= {}

    gare_data["id"] = gare["recordid"]

    for field in field_list:
        try: 
            gare_data[field[0]] = gare["fields"][field[1]]
        except KeyError:
            gare_data[field[0]]=None

   
    session.add(Gare(**gare_data))
    session.commit()
    # try:
    #     session.commit()
    # except:
    #     print(gare_data["code_uic"])

# Données des fréquentations

In [ ]:
from create_model import Frequentation
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine


engine = create_engine('sqlite:///db.sqlite')
Session = sessionmaker(bind=engine)
session = Session()

url_game = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=frequentation-gares&q=&sort=nom_gare&rows=10000"
my_request = requests.get(url_game)


field_list = [
    ["code_uic", "code_uic_complet"],
    ["code_postal", "code_postal"],
    ["total_voyageurs_2016", "total_voyageurs_2016"],
    ["total_voyageurs_2017", "totalvoyageurs2017"],
    ["total_voyageurs_2018", "total_voyageurs_2018"],
    ["total_voyageurs_2019", "total_voyageurs_2019"],
    ["total_voyageurs_2020", "total_voyageurs_2020"],
    ["total_voyageurs_2021", "total_voyageurs_2021"]
]

for frequentation in my_request.json()["records"]:
    frequentation_data= {}

    frequentation_data["id"] = frequentation["recordid"]

    for field in field_list:
        if field[0] =="code_uic":
            try: 
                frequentation_data[field[0]] = "00" + str(frequentation["fields"][field[1]])
            except KeyError:
                frequentation_data[field[0]]=None
        else:
            try: 
                frequentation_data[field[0]] = frequentation["fields"][field[1]]
            except KeyError:
                frequentation_data[field[0]]=None

   
    session.add(Frequentation(**frequentation_data))
    session.commit()


# Données météorologiques

In [ ]:
# dep_with_gare= [ '75', '10', '54', '88', '51', '68', '90', '67', '57', '59',
#        '95', '60', '62', '80', '84', '13', '27', '76', '14', '61', '35',
#        '29', '56', '53', '44', '49', '17', '85', '45', '37', '41', '86',
#        '18', '33', '16', '24', '47', '87', '36', '31', '11', '65', '64',
#        '34', '58', '03', '21', '25', '69', '71', '42', '63', '73', '01',
#        '74', '38', '83', '06', '26', '05', '30', '66']

In [ ]:
# def specific_url_temp(dep,annee):
#     URL = "https://public.opendatasoft.com"
#     ressource = "/api/records/1.0/search/?dataset=donnees-synop-essentielles-omm&q=&rows=10000&sort=date"
#     month_format = f"&refine.code_dept={dep}&refine.date={annee}"
#     return URL + ressource + month_format


In [ ]:
# from create_model import TemperatureHeure
# from sqlalchemy.orm import sessionmaker
# from sqlalchemy import create_engine


# engine = create_engine('sqlite:///db.sqlite')
# Session = sessionmaker(bind=engine)
# session = Session()


# field_list = [
#     ["date", "date"],
#     ["temperature", "tc"],
#     ["region", "nom_reg"],
#     ["region_code", "code_reg"],
#     ["departement", "nom_dept"],
#     ["departement_code", "code_dep"],
#     ["commune", "libgeo"],
#     ["commune_code", "codegeo"],
# ]

# for dep in dep_with_gare:
#     for annee in [2016,2017,2018,2019,2020,2021]:
#         my_request = requests.get(specific_url_temp(dep,annee))
#         print(f"request réalisée pour , {annee}, {dep}")
#         for temp in my_request.json()["records"]:
            
#             temp_data= {}

#             for field in field_list:
#                 try: 
#                     temp_data[field[0]] =temp["fields"][field[1]]
#                 except KeyError:
#                     temp_data[field[0]]=None


#             session.add(TemperatureHeure(**temp_data))
#         session.commit()


In [ ]:
import pandas as pd
df_temp= pd.read_csv("agregated_temperature.csv")
df_temp.to_sql("Temperature",con=engine,if_exists="replace")